# Check observables given a results file from a TRSL simulation

The purpose of this notebook is to compare some results of the TRSL model against real world observables in order to test the calibration.

We are checking:

1. Protein synthesis rate
2. x

In [1]:
import cPickle as pkl
import random
import numpy as np
import pandas as pd

import

matplotlib.use("pgf")
pgf_with_rc_fonts = {
    "font.family": "serif",
    "font.serif": [u'Adobe Caslon Pro'],                   # use latex default serif font
    "font.sans-serif": ["DejaVu Sans"], # use a specific sans-serif font
}
matplotlib.rcParams.update(pgf_with_rc_fonts)
%matplotlib inline
matplotlib.style.use('ggplot')

import matplotlib.pyplot as plt

from statsmodels.nonparametric.smoothers_lowess import lowess

NameError: name 'matplotlib' is not defined

In [ ]:
#filepath = "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20151120_2051_3600s.p"
#filepath = "../../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160906_1752_3600s.p"
filepath = "../../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20170331_2025_200000_ribosomes_3600s.p"

The following assumes the results file is readable (otherwise replace pypy specifics):

In [ ]:
results = pkl.load(open(filepath))

## 1. Protein synthesis rate

Define protein names in results:

In [ ]:
protnames = results['timecourses'].keys()
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'peptide_bonds']

for fragment in blacklist:
    protnames = [name for name in protnames if fragment not in name]

print len(protnames)

Plot some randomly selected timecourses:

In [ ]:
size = 4
protnames_plot = np.array(random.sample(protnames, size * size))
protnames_plot.shape = (size, size)

In [ ]:
fig, ax = plt.subplots(size, size, figsize=(20, 20))

for i in range(size):
    for j in range(size):
        xseries = results['timerange']
        yseries = results['timecourses'][protnames_plot[i][j]]
        # oddly, sometimes the simulation sometimes 'loses' one data point
        length = min(len(xseries), len(yseries))
        ax[i, j].plot(xseries[:length], yseries[:length])
        ax[i, j].set_xticks(np.arange(min(xseries), max(xseries)+1, 1000.0))
        ax[i, j].set_title(protnames_plot[i][j], fontdict={'fontsize': 15, 'fontweight': 'bold'})
        ax[i, j].tick_params(axis='both', which='major', labelsize=15)